# Optical Flow with Convolution Neural Networks

The paper's version of optical flow produces a vector <-x, +x, -y, +y, -z, +z>.
Our result vector needs to change in track_object(frame1, frame2, x, y, w, h, steps) - currently the displacement should be `[x,y]`, so we'll just need to do some conditional wizardry and expand it out.

*NOTE:* Prior to running any of this, a virtual environment is needed. Once in the venv, run `pip install -r requirements.txt` to prepare all of the required libraries.

In [1]:
import numpy as np
flow = np.array([-2,3])
final_flow = np.array([0, 0, 0, 0, 0, 0])

if flow[0] < 0:
    final_flow[0] = abs(flow[0])
elif flow[0] > 0:
    final_flow[1] = abs(flow[0])

if flow[1] < 0:
    final_flow[2] = abs(flow[1])
elif flow[1] > 0:
    final_flow[3] = abs(flow[1])
final_flow

array([2, 0, 0, 3, 0, 0])

The videos used are all releaseed under the Creative Commons Attribution 3.0 license.
We can generate the list of all files in the training directory (in this case, a scene from "Big Buck Bunny") as follows:

In [2]:
import os
file_list = os.listdir("bunny")

We'll need to add the directory name to the beginning of each file:

In [3]:
for x in range(len(file_list)):
    file_list[x] = 'bunny/' + file_list[x]

What if we use the "Good Features to Track" algorithm to compute the optical flow with OpenCV?
The following cell's contents are based on an example found at [https://www.geeksforgeeks.org/python-opencv-optical-flow-with-lucas-kanade-method/](https://www.geeksforgeeks.org/python-opencv-optical-flow-with-lucas-kanade-method/)

In [ ]:
import numpy as np
import cv2
import imageio

def run_network(frame_list):
    recording = []
    for x in range(len(frame_list)):
        recording.append(imageio.imread(frame_list[x])[
            :, :, :3].astype(np.float32) / 255.0)
    return recording

cap = run_network(file_list)

feature_params = dict(maxCorners=100,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)

lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))


old_frame = cap.pop(0)
p0 = cv2.goodFeaturesToTrack(old_frame, mask=None, **feature_params)

while len(cap) > 0:
    frame = cap.pop(0)

    p1, st, err = cv2.calcOpticalFlowPyrLK(old_frame,
                                           frame,
                                           p0, None,
                                           **lk_params)

    good_new = p1[st == 1]
    good_old = p0[st == 1]

    old_gray = frame.copy()
    p0 = good_new.reshape(-1, 1, 2)

cv2.destroyAllWindows()
cap.release()


Preparing the dataset will use the `prepare_dataset` function in the `optical_flow.py` file. The signature for this function is:
`prepare_dataset(files_list, result_file, x, y, w, h, steps=5)`
The bounding box should be determined for the scene, and consists of the x,y,w,h parameters.

In [4]:
from optical_flow import prepare_dataset
prepare_dataset(file_list, "train-data.csv", 124, 158, 254, 233, steps=5)

KeyboardInterrupt: 

To train the neural net, we'll use the Lkvf file.

In [ ]:
from Lkvf import train
train()

Once we're trained up, we can test the neural network on arbitary video capture. We'll use the same video we trained on to try and generate a list of flow vectors.

In [ ]:
from RunLKNN import run_network

run_network(file_list)